In [1]:
from pymatgen.entries.computed_entries import Composition
from pymatgen.core.periodic_table import Element
from pymatgen.core.structure import Structure
from pymatgen.transformations.standard_transformations import SubstitutionTransformation
from itertools import product
import pandas as pd
from pymatgen.ext.matproj import MPRester
import os
from math import gcd
from pymatgen.analysis.local_env import BVAnalyzer
from functools import reduce
import glob
import shutil


/Users/tolstop/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h3>Create a dictionary for possible metals' oxidation states</h3>

In [2]:
def get_possible_oxidation_states(element_symbol):
    element = Element(element_symbol)
    return [str(ox) for ox in element.oxidation_states]

# Create a dictionary for possible metals' oxidation states 
oxidation_state_dict = {}
elements_to_check = ["Li", "Be", "Na", "Mg", "Al", "K", "Ca", "Sc", "Ti", "V", "Cr",
                  "Mn", "Fe", "Ni", "Co", "Cu", "Zn", "Ga", "Ge", 'As', "Rb", "Sr", "Y", "Zr", "Nb", "Mo",
                  "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", 'Sb', 'Te', "Cs", "Ba", "La", "Hf", "Ta",
                  "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi"]
for element in elements_to_check:
    oxidation_states = get_possible_oxidation_states(element)
    for state in oxidation_states:
        oxidation_state_dict.setdefault(state, []).append(element)


<h3>Elemental substitution based on oxidation states of metals which are in the structure</h3>
Initially, elemental substituion is made for compositions which for pairs between themselves. However, sometimes BVAnalyzer can't define oxidation states for a composition. If among chemical space M1-M2-S there is only one composition for which it is possible to define oxidation states, than it does not do elemental substitution for this composition as well. So new structures are formed only from those compositions which already have a redox pair

In [3]:
def process_site(original_structure, structure_analyzed, oxidation_states):
    for site_name, site in zip(original_structure, structure_analyzed.sites):
        element_symbol = str(site_name.specie)

        oxidation_states[element_symbol] = oxidation_states.get(element_symbol, set())
        oxidation_states[element_symbol].add(str(site.specie.oxi_state))

def extract_oxidation_states(original_structure):
    bva = BVAnalyzer()
    structure_analyzed = bva.get_oxi_state_decorated_structure(original_structure)
    oxidation_states = {}

    process_site(original_structure, structure_analyzed, oxidation_states)

    return oxidation_states

def find_possible_metals(result, oxidation_state_dict):
    # Get the oxidation states from the result dictionary
    oxidation_states_from_result = [result[key] for key in result]
    # Find the intersections of oxidation states
        
    m1_possible_metals = list(find_possible_metal_list(oxidation_states_from_result[0], oxidation_state_dict))
    m2_possible_metals = list(find_possible_metal_list(oxidation_states_from_result[1], oxidation_state_dict))

    return m1_possible_metals, m2_possible_metals

def find_possible_metal_list(oxidation_states_from_result, oxidation_state_dict):
    # Convert the list of sets to a list of keys
    keys_list = [next(iter(s)) for s in oxidation_states_from_result]
    # Find the set of intersections for all keys in keys_list
    possible_metal_set = [set(oxidation_state_dict[key]) for key in keys_list]
    intersection_result_reduce = reduce(lambda x, y: x.intersection(y), possible_metal_set)
    return intersection_result_reduce

def without_errors(structure, parent_formula):
    without_errors = 0
    for original_structure, original_formula in zip(structure, parent_formula):
        composition = Composition(original_structure.composition)
        element_composition = composition.element_composition.as_dict()
        m1_symbol = list(element_composition.keys())[0]
        m2_symbol = list(element_composition.keys())[1]

        # Extract M1 and M2 oxidation states
        metal_1 = str(composition.elements[0])
        metal_2 = str(composition.elements[1])  
        try: 
            result = extract_oxidation_states(original_structure)

            without_errors += 1
            if without_errors >= 2:
                break

        except Exception as e:
            print(f"without_errors_checking: {original_formula}: {e}")
        
    return without_errors


    
def generate_substituted_structures(original_structure, m1_symbol, m2_symbol,
                                    m1_possible_metals, m2_possible_metals,
                                    original_formula):
    all_substituted_structures = []

    for m1 in m1_possible_metals:
        for m2 in m2_possible_metals:
            if m1 != m2:
                # Create a substitution mapping for m1 and m2
                substitution_mapping = {m1_symbol: f'{m1}', m2_symbol: f'{m2}'}
                    # Apply the substitution transformation to the original structure
                substitution_transformation = SubstitutionTransformation(substitution_mapping)
                substituted_structure = substitution_transformation.apply_transformation(original_structure)

                # Add to the list of substituted structures
                all_substituted_structures.append({
                    'pretty_formula': substituted_structure.composition.reduced_formula,
                    'structure': substituted_structure,
                    'parent_formula': original_formula,
                    'parent_structure': original_structure
                })

    df_substituted_structures = pd.DataFrame(all_substituted_structures)
    return df_substituted_structures

def get_unique_elements(formula):
    composition = Composition(formula)
    return ''.join(sorted(set(str(element) for element in composition.elements)))

def sulfur_amount(row):
    return row['element_amounts']['S']

def calculate_m1_m2_ratio(row):
    row['element_amounts'].pop('S')
    m1, m2 = row['element_amounts'].values()
    ratio = m1 / m2
    gcd_value = gcd(int(m1), int(m2))   
    return ratio, gcd_value


In [4]:
def get_df_substituted_structures(df):
        
    df['unique_elements'] = df['pretty_formula'].apply(get_unique_elements)
    df['composition'] = df['pretty_formula'].apply(Composition)
    df['element_amounts'] = df['composition'].apply(lambda comp: comp.get_el_amt_dict())
    df['sulfur_amount'] = df.apply(lambda row: sulfur_amount(row), axis=1)
    df[['M1_M2_ratio', 'gcd']] = df.apply(lambda row: calculate_m1_m2_ratio(row), axis=1, result_type="expand")
    df['Sn_normalized'] = df['sulfur_amount']/df['gcd']
    
    chemspace = set(df['unique_elements'].to_list())   
    #create a list of unique values of compositions    
    for subset in chemspace:
        ratios = df.loc[df['unique_elements'] == subset, 'M1_M2_ratio'].unique()
        for ratio in ratios:
            df_small = df.loc[(df['unique_elements'] == subset) & 
                                   (df['M1_M2_ratio'] == ratio)]

            if len(df_small.Sn_normalized.unique()) > 1: 
                parent_formula = df_small['pretty_formula'].to_list()
                structure = df_small['structure'].to_list()
                without = without_errors(structure, parent_formula)

                if without >= 2 :

                    for original_structure, original_formula in zip(structure, parent_formula):
                        composition = Composition(original_structure.composition)
                        element_composition = composition.element_composition.as_dict()

                        m1_symbol = list(element_composition.keys())[0]
                        m2_symbol = list(element_composition.keys())[1]

                        try:
                            result = extract_oxidation_states(original_structure)
                            removed_states = result.pop('S')
                            m1_possible_metals, m2_possible_metals = find_possible_metals(result, oxidation_state_dict)
                            df_substituted_structures = generate_substituted_structures(original_structure, m1_symbol, m2_symbol,
                                                    m1_possible_metals, m2_possible_metals,
                                                    original_formula)
                            print(f"df for {original_formula}")
                            df_substituted_structures.to_csv(f'from_{original_formula}.csv')
                        except Exception as e:
                            print(f"will not create df for {original_formula}: {type(e).__name__} - {str(e)}")
                            continue


In [5]:
# Query materials project
mp_ids = [
    'mp-17823', 'mp-1347880', 'mp-768204', 'mp-27802', 'mp-1224980', 'mp-1199702', 'mp-756094', 
    'mp-1212802', 'mp-1218773', 'mp-7499', 'mp-3102', 'mp-1407214', 'mp-1204783', 'mp-27660', 
    'mp-1147661', 'mp-5038', 'mp-5702', 'mp-1218028', 'mp-675748', 'mp-1183370', 'mp-1366511', 
    'mp-1342988', 'mp-675124', 'mp-1120784', 'mp-2240411', 'mp-753722', 'mp-1215190', 'mp-1395289', 
    'mp-28859', 'mp-22035', 'mp-1224893', 'mp-1194339', 'mp-653954', 'mp-1219007', 'mp-1078387', 
    'mp-776356', 'mp-1246794', 'mp-1225005', 'mp-1199798', 'mp-1045397', 'mp-3075', 'mp-1218051', 
    'mp-1192902', 'mp-28701', 'mp-3797', 'mp-768163', 'mp-1206827', 'mp-17691', 'mp-8378', 'mp-27146',
    'mp-14578', 'mp-1045420', 'mp-1212593', 'mp-984714', 'mp-9047', 'mp-1225004', 'mp-1369134', 'mp-753429',
    'mp-28270', 'mp-1348032', 'mp-776405', 'mp-3345', 'mp-505747', 'mp-1344076', 'mp-1213664', 'mp-1202711', 
    'mp-1374814', 'mp-767553', 'mp-29301', 'mp-1194266', 'mp-646878', 'mp-756427', 'mp-558762', 'mp-7792', 
    'mp-753833', 'mp-10167', 'mp-1353425', 'mp-1410970', 'mp-28011', 'mp-1177470', 'mp-5818', 'mp-35860', 
    'mp-28308', 'mp-767409', 'mp-1226309', 'mp-1408976', 'mp-1368283', 'mp-1103174', 'mp-9910', 'mp-562726', 
    'mp-1390919', 'mp-849803', 'mp-7937', 'mp-4194', 'mp-1400392', 'mp-1390164', 'mp-1215189', 'mp-676562', 
    'mp-561639', 'mp-1333226', 'mp-1193319', 'mp-561620', 'mp-36254', 'mp-1226240', 'mp-1219053', 'mp-1079885',
    'mp-1221502', 'mp-1227043', 'mp-1048312', 'mp-36100', 'mp-1394404', 'mp-1221529', 'mp-1194545', 'mp-1199244', 
    'mp-1045503', 'mp-1443978', 'mp-504884', 'mp-1189757', 'mp-1341750', 'mp-760375', 'mp-5862', 'mp-1178038',
    'mp-9781', 'mp-1045467', 'mp-2224818', 'mp-1193673', 'mp-680170', 'mp-540724', 'mp-1216612', 'mp-8965', 
    'mp-18008', 'mp-2365577', 'mp-766422', 'mp-2210614', 'mp-15012', 'mp-1337045', 'mp-1224971', 'mp-7077',
    'mp-15013', 'mp-725977', 'mp-1213632', 'mp-15011', 'mp-756394', 'mp-1443364', 'mp-9538', 'mp-767516',
    'mp-9615', 'mp-7928', 'mp-1045498', 'mp-1220404', 'mp-3592', 'mp-1226064', 'mp-22982', 'mp-12307', 
    'mp-1072589', 'mp-1219470', 'mp-1218951', 'mp-18003', 'mp-1096842', 'mp-1323643', 'mp-4351', 'mp-1104162', 
    'mp-767527', 'mp-769205', 'mp-1216903', 'mp-9622', 'mp-1210002', 'mp-562480', 'mp-8890', 'mp-559356', 
    'mp-753632', 'mp-1227354', 'mp-1217134', 'mp-674355', 'mp-1217959', 'mp-1226465', 'mp-29411', 'mp-1407369',
    'mp-2217928', 'mp-1330582', 'mp-1147769', 'mp-753720', 'mp-1215454', 'mp-1104922', 'mp-1211069', 'mp-1205388',
    'mp-1346425', 'mp-1400406', 'mp-1222701', 'mp-756316', 'mp-1217148', 'mp-1193111', 'mp-1176510', 'mp-1188095',
    'mp-999267', 'mp-675396', 'mp-753776', 'mp-621960', 'mp-774745', 'mp-753863', 'mp-8946', 'mp-768229', 
    'mp-5704', 'mp-2232752', 'mp-1080466', 'mp-1199053', 'mp-8393', 'mp-1226336', 'mp-504814', 'mp-1219055', 
    'mp-22739', 'mp-1216184', 'mp-541379', 'mp-1215520', 'mp-7543', 'mp-1079869', 'mp-2226962', 'mp-774762', 
    'mp-557059', 'mp-1176579', 'mp-30294', 'mp-17228', 'mp-14089', 'mp-1223513', 'mp-1220398', 'mp-2210632',
    'mp-1384673', 'mp-1193894', 'mp-28809', 'mp-1226487', 'mp-12091', 'mp-1372476', 'mp-12181', 'mp-13740', 
    'mp-768139', 'mp-17154', 'mp-1224903', 'mp-559551', 'mp-1329064', 'mp-1106085', 'mp-1247743', 
    'mp-1070151', 'mp-15148', 'mp-1215915', 'mp-1218077', 'mp-652663', 'mp-1096836', 'mp-1177695', 'mp-574909'
]
mpr = MPRester('Ax4Q4nqF0JzF4nwrlYW1SruccPcMe9BA')
data = mpr.summary.search(
    material_ids=mp_ids,
    fields=[
        "formula_pretty",
        'material_id',
        "structure"    
    ]
)

mpid_dict = {dat.material_id: [dat.formula_pretty,
                               dat.structure] for dat in data}
columns=["pretty_formula",'structure']


df_new = pd.DataFrame.from_dict(mpid_dict,orient='index', columns=columns)
df_new = df_new.rename_axis('MP_id').reset_index()


Retrieving SummaryDoc documents: 100%|██████████████████| 245/245 [00:00<00:00, 888162.90it/s]


In [6]:
def main():
    get_df_substituted_structures(df_new)
if __name__ == "__main__":
    main()

df for Li3VS4
df for Li3VS3
df for Li2V2S5
df for LiVS3
df for LiVS2
df for Na2B2S5
df for NaBS2
df for Ba(FeS2)2
df for BaFe2S3
without_errors_checking: K2Ni3S2: Valences cannot be assigned!
df for K2B2S7
df for KBS2
df for ZnCo2S5
df for Zn(CoS2)2
df for FeCoS2
df for FeCoS4
df for Zn(FeS2)2
df for ZnFe2S5
df for K2Sn2S5
df for KSnS4
df for KSnS2
without_errors_checking: K2NbS7: Valences cannot be assigned!
df for Tl3B3S10
df for TlBS2
df for TlBS3
df for FeNiS4
df for FeNiS2
df for Ag3AsS3
df for Ag3AsS4
df for AgAsS2
will not create df for AgAsS: KeyError - '-'
df for Co(NiS2)2
df for Co3(Ni3S4)2
df for Co3Ni3S8
df for CoNiS4
df for Li2CrS3
df for Li2CrS4
df for Cu2GeS4
df for Cu2GeS3
df for LaTmS2
df for LaTmS3
df for Li2BiS2
df for Li2BiS3
df for Li4Bi2S7
df for Li3BiS3
df for Li3BiS4
df for CuBiS2
df for Cu4Bi4S9
df for Zn(NiS2)2
df for ZnNi2S5
df for Zn(CrS2)2
df for ZnCr2S5
df for SnPbS2
df for SnPbS3
df for Ba2GaS4
df for Ba4Ga2S7
df for MgCo2S5
df for Mg(CoS2)2
df for Ta2NiS

In [7]:
# Use glob to get a list of file paths for all CSV files starting with 'from_'
file_paths = glob.glob("from_*.csv")


# Initialize an empty list to store DataFrames
dfs = []

# Iterate through each file path
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames in the list into one big DataFrame
big_dataframe = pd.concat(dfs, ignore_index=True)

print(f'Total amount of compounds is {len(big_dataframe)}')
big_dataframe.to_csv('all_new.csv')

Total amount of compounds is 235162


<h3>Reorganising .csv files, preparing for BOWSR relaxation </h3>
Removing from hypothetical compositions those from which they were actually made of. Filter for final df only those which have a redox pair in the df. Organize to the folders for performing calculations

In [8]:
big_dataframe = pd.read_csv('all_new.csv')
big_dataframe

,Unnamed: 0.1,Unnamed: 0,pretty_formula,structure,parent_formula,parent_structure
0,0,0,Zn3RuS3,Full Formula (Zn24 Ru8 S24)\nReduced Formula: ...,Tl3AsS3,Full Formula (Tl24 As8 S24)\nReduced Formula: ...
1,1,1,Zn3CoS3,Full Formula (Zn24 Co8 S24)\nReduced Formula: ...,Tl3AsS3,Full Formula (Tl24 As8 S24)\nReduced Formula: ...
2,2,2,Zn3InS3,Full Formula (Zn24 In8 S24)\nReduced Formula: ...,Tl3AsS3,Full Formula (Tl24 As8 S24)\nReduced Formula: ...
3,3,3,Mn(ZnS)3,Full Formula (Mn8 Zn24 S24)\nReduced Formula: ...,Tl3AsS3,Full Formula (Tl24 As8 S24)\nReduced Formula: ...
4,4,4,Sc(ZnS)3,Full Formula (Sc8 Zn24 S24)\nReduced Formula: ...,Tl3AsS3,Full Formula (Tl24 As8 S24)\nReduced Formula: ...
...,...,...,...,...,...,...
235157,235157,1352,Ga(OsS2)2,Full Formula (Ga2 Os4 S8)\nReduced Formula: Ga...,Mg(TiS2)2,Full Formula (Mg2 Ti4 S8)\nReduced Formula: Mg...
235158,235158,1353,Ta2GaS4,Full Formula (Ta4 Ga2 S8)\nReduced Formula: Ta...,Mg(TiS2)2,Full Formula (Mg2 Ti4 S8)\nReduced Formula: Mg...
235159,235159,1354,V2GaS4,Full Formula (V4 Ga2 S8)\nReduced Formula: V2G...,Mg(TiS2)2,Full Formula (Mg2 Ti4 S8)\nReduced Formula: Mg...
235160,235160,1355,Ga(AuS2)2,Full Formula (Ga2 Au4 S8)\nReduced Formula: Ga...,Mg(TiS2)2,Full Formula (Mg2 Ti4 S8)\nReduced Formula: Mg...


In [9]:
filtered_big_dataframe = big_dataframe[~big_dataframe['pretty_formula'].isin(df_new['pretty_formula'])]
len(filtered_big_dataframe)

232440

In [11]:

# Specify the directory for the new folder
new_folder_path = "new_materials/"

# Create the new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Define a function to get unique elements from 'pretty_formula'
def get_unique_elements(formula):
    composition = Composition(formula)
    return ''.join(sorted(set(str(element) for element in composition.elements)))

def sulfur_amount(row):
    return row['element_amounts']['S']

def calculate_m1_m2_ratio(row):
    row['element_amounts'].pop('S')
    m1, m2 = row['element_amounts'].values()
    ratio = m1 / m2
    gcd_value = gcd(int(m1), int(m2))   
    return ratio, gcd_value
sum_len=0
# Apply the function to create a new column 'unique_elements'
filtered_big_dataframe['unique_elements'] = filtered_big_dataframe['pretty_formula'].apply(get_unique_elements)

# Iterate through unique combinations
for combination in filtered_big_dataframe['unique_elements'].unique():
    # Create a new DataFrame for the current combination
    subset_dataframe = filtered_big_dataframe[filtered_big_dataframe['unique_elements'] == combination].copy()

    # Drop the 'unique_elements' column if you don't need it in the final CSV
    subset_dataframe['composition'] = subset_dataframe['pretty_formula'].apply(Composition)
    subset_dataframe['element_amounts'] = subset_dataframe['composition'].apply(lambda comp: comp.get_el_amt_dict())


    subset_dataframe['sulfur_amount'] = subset_dataframe.apply(lambda row: sulfur_amount(row), axis=1)
    subset_dataframe[['M1_M2_ratio', 'gcd']] = subset_dataframe.apply(lambda row: calculate_m1_m2_ratio(row), axis=1, result_type="expand")
    subset_dataframe['Sn_normalized'] = subset_dataframe['sulfur_amount']/subset_dataframe['gcd']
    composition_set = set(subset_dataframe['M1_M2_ratio'].to_list()) #create a list of unique values of compositions
    for material in composition_set:

        df_small = subset_dataframe.loc[((subset_dataframe['M1_M2_ratio'] == material))]
        if len(df_small.Sn_normalized.unique()) == 1: 
            subset_dataframe = subset_dataframe[subset_dataframe['M1_M2_ratio'] != material] 
            
    subset_dataframe.reset_index(inplace=True, drop=True)
    subset_dataframe.drop(['Unnamed: 0', 'composition', 'element_amounts', 'sulfur_amount', 'gcd', 'unique_elements'], axis=1, inplace=True)

    # Save the DataFrame to a CSV file in the new folder
    file_name = f"subset_{combination}.csv"
    file_path = os.path.join(new_folder_path, file_name)
    subset_dataframe.to_csv(file_path, index=False)

#     print(f"Saved {combination}") 
#     print(len(subset_dataframe))
    sum_len += len(subset_dataframe)
print(f'total amount of final compositions for BOWSR is {sum_len}')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


total amount of final compositions for BOWSR is 231378


In [12]:
csv_directory = 'new_materials/'
folders = ['folder1', 'folder2', 'folder3', 'folder4', 'folder5']

# Create folders if they don't exist
for folder in folders:
    folder_path = os.path.join(csv_directory, folder)
    os.makedirs(folder_path, exist_ok=True)

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(csv_directory) if file.endswith('.csv')]

# Calculate the number of files to put in each folder
batch_size = len(csv_files) // len(folders)

# Distribute CSV files into folders
for i, folder in enumerate(folders):
    start_index = i * batch_size
    end_index = (i + 1) * batch_size
    if i == len(folders) - 1:
        end_index = len(csv_files)  # Include remaining files in the last folder
    files_to_move = csv_files[start_index:end_index]
    for file_name in files_to_move:
        src = os.path.join(csv_directory, file_name)
        dst = os.path.join(csv_directory, folder, file_name)
        shutil.move(src, dst)

print("CSV files organized into folders successfully.")

CSV files organized into folders successfully.
